In [1]:
import openpyxl
import pandas as pd
import os
import datetime
#from IPython.display import display

In [2]:
def print_full(x):
    df = pd.DataFrame(x)
    pd.set_option('display.max_rows', 2000)
    pd.set_option('display.max_columns', 20)
    display(df)

### import and export excel 

In [3]:
excelname= "ma"
sheetname = "10ma"

wb = openpyxl.load_workbook(str(excelname)+".xlsx")
ws = wb[str(sheetname)]

dataset = []

i = 1
while(ws.cell(row=i, column=1).value!=None):
    row = []
    j = 1
    while(ws.cell(row=i, column=j).value!=None):
        row.append(ws.cell(row=i, column=j).value)
        j = j +1
    dataset.append(row)
    i = i + 1

In [4]:
trade =[[0 for x in range(9)] for y in range(len(dataset))]
trade[0][0] = "date"
trade[0][1] = "buy"
trade[0][2] = "hold"
trade[0][3] = "sell"
trade[0][4] = "number"
trade[0][5] = "value"
trade[0][6] = "period_return"
trade[0][7] = "closeprice"
trade[0][8] = "10ma"

In [5]:
for i in range(1, len(trade)):
    trade[i][0] = dataset[i][0]
    trade[i][7] = float(dataset[i][7])
    
for i in range(10, len(trade)):
    temp = 0
    for j in range(0, 10):
        temp = temp + trade[i-j][7]
    trade[i][8] = temp / 10

In [6]:
for i in range(10, len(trade)):
    if(trade[i][7] > trade[i][8])and(trade[i-1][2]==0):
        trade[i][1] = 1
        trade[i][2] = 1
        trade[i][3] = 0
    elif(trade[i][7] < trade[i][8])and(trade[i-1][2]==1):
        trade[i][1] = 0
        trade[i][2] = 0
        trade[i][3] = 1
    elif(trade[i-1][2]>0):
        trade[i][1] = 0
        trade[i][2] = 1
        trade[i][3] = 0

In [7]:
initial = 10000
for i in range(10, len(trade)):
    if(trade[i][1] == 1)and(trade[i][2]==1)and(trade[i][3] == 0):
        trade[i][4] = initial/trade[i][7]
        trade[i][5] = trade[i][4] * trade[i][7]
        trade[i][6] = round((trade[i][4] * trade[i][7] - initial)/initial, 4)
    elif(trade[i][1] == 0)and(trade[i][2]==0)and(trade[i][3] == 1):
        trade[i][4] = 0
        trade[i][5] = trade[i-1][4] * trade[i][7]
        trade[i][6] = round((trade[i-1][4] * trade[i][7] - initial)/initial, 4)
    elif(trade[i][1] == 0)and(trade[i][2]==1)and(trade[i][3] == 0):
        trade[i][4] = trade[i-1][4]
        trade[i][5] = trade[i][4] * trade[i][7]
        trade[i][6] = round((trade[i][4] * trade[i][7] - initial)/initial, 4)

In [8]:
for i in range(0, len(trade)):
    for j in range(0, len(trade[1])):
        ws.cell(row=i+1, column=j+1).value= trade[i][j]
wb.save('ma.xlsx')